# Introduction: single-task optimization

We start with a common test function for optimization known as [Himmelblau's function](https://en.wikipedia.org/wiki/Himmelblau%27s_function), which gives us a single optimization task (find the inputs that minimize the function).

In [ ]:
%run -i ../../../examples/prepare_bluesky.py

import bloptools
from bloptools.experiments.tests import himmelblau

boa = bloptools.bayesian.Agent(
                            dofs=himmelblau.dofs, # things which we move around
                            bounds=himmelblau.bounds, # where we're allowed to move them
                            tasks=[himmelblau.MinHimmelblau], # tasks for the optimizer
                            acquisition=himmelblau.acquisition, # a plan to sample data
                            digestion=himmelblau.digestion, # how to process the acquisition into task data
                            db=db, # a databroker instance
                            )

RE(boa.initialize(init_scheme='quasi-random', n_init=32))

We initialized the GP with the "quasi-random" strategy, as it doesn't require any prior data. We can view the state of the optimizer's posterior of the tasks over the input parameters:

In [ ]:
boa.plot_tasks()

We can also the agent's posterior about the probability of goodness over the parameters:

In [ ]:
boa.plot_constraints()

We want to learn a bit more, so we can ask the agent where to sample based off of some strategy. Here we use the "esti" strategy, which maximizes the expected sum of tasks improvement.

We can ask the agent to "route" them using ``ortools``, so that we can sample them more quickly if it requires us to e.g. move motors.

In [ ]:
X_to_sample = boa.ask(strategy='esti', n=16, optimize=True, route=True)
plt.scatter(*X_to_sample.T)
plt.plot(*X_to_sample.T)

Let's tell the agent to learn a bit more (it will direct itself):

In [ ]:
RE(boa.learn(strategy='esti', n_iter=1, n_per_iter=4))
boa.plot_tasks()

The agent has updated its model of the tasks, including refitting the hyperparameters. Continuing:

In [ ]:
RE(boa.learn(strategy='esti', n_iter=4, n_per_iter=4))
boa.plot_tasks()

Eventually, we reach a point of saturation where no more improvement takes place:

In [ ]:
RE(boa.learn(strategy='esti', n_iter=4, n_per_iter=4))
boa.plot_tasks()